In [2]:
import pandas as pd
import requests

#para normalizar fechas
import datetime
#para obtener fechas
import time

In [64]:
# Obtenemos información a partir de la API

url = f'https://www.ngdc.noaa.gov/hazel/hazard-service/api/v1/tsunamis/events?country=USA&maxYear=2023&minYear=2010'
dt_usa_prev_Mar = pd.DataFrame(requests.get(url).json()['items'])

url = f'https://www.ngdc.noaa.gov/hazel/hazard-service/api/v1/tsunamis/events?country=JAPAN&maxYear=2023&minYear=2010'
dt_jap_prev_Mar = pd.DataFrame(requests.get(url).json()['items'])

url = f'https://www.ngdc.noaa.gov/hazel/hazard-service/api/v1/tsunamis/events?country=CHILE&maxYear=2023&minYear=2010'
dt_chi_prev_Mar = pd.DataFrame(requests.get(url).json()['items'])

In [65]:
def mar(df):
    
    df=df.fillna(0)
    df['time'] = pd.to_datetime(df[['year','month','day','hour','minute','second']],format='%Y/%m/%d %H:%M:%S')
    df['time'] =df['time'].apply(lambda x: x.strftime('%Y/%m/%d %H:%M:%S'))
    df.rename(columns={'eqMagnitude':'mag', 'eqDepth':'deepth', 'latitude':'lat', 'longitude':'lng',
    'locationName':'place', 'maxWaterHeight':'mxwatheight'}, inplace=True)


    df['mxwatheight']  = df.mxwatheight.astype('float64')
    df.mag=df.mag.astype('float64')
    df.lat=df.lng.astype('float64')
    df.lat=df.lat.astype('float64')
    df.deepth=df.deepth.astype('float64')

    # Filtramos el dataframe con los datos relevantes.
    df = df[['country','mxwatheight','place', 'time', 'mag', 'lng',
    'lat', 'deepth' ]]
    return (df)    

In [68]:
dt_usa_Mar=mar(dt_usa_prev_Mar)
dt_chi_Mar=mar(dt_chi_prev_Mar)
dt_jap_Mar=mar(dt_jap_prev_Mar)